In [88]:
import numpy as np
import importlib
import Layout
import Greedy

import copy 

#Reset libreria
importlib.reload(px)
importlib.reload(Greedy)

def showtime2(path):
    layout=px.read(path)
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    seleccionada = SSRC(layout)
 
    while seleccionada != -1:
        #print("SELECCIONADA:", seleccionada)
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        #print(rank)

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        #for s in layout.stacks:
        #    print(s)
        #print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)


        SFpp(layout,seleccionada,rank,nb_unsorted_elements)
        #for s in layout.stacks:
        #    print(s)
        #print("\n")

        #Se vuelve a seleccionar una columna
        seleccionada = SSRC(layout)

    #print("SELECCIONADA:", seleccionada)
    print(len(layout.moves),"\t",layout.unsorted_stacks)
    return len(layout.moves)
    #print()

In [58]:
#SELECT SEMI RANDOM COLUMN
def SSRC(layout):
    
    size=len(layout.stacks)
    best=-1000000000
    select=-1
    
    posibles = []

    for i in range (0,size):

        aux = layout.is_sorted_stack(i)

        if aux == False:
            posibles.append(i)

    #Me di cuenta que con el que mayor problemas tiene tenemos menos problemas 
    #(necesita menos espacio al contrario del que tenga menos problemas)

    #Si no hay posibles se retorna -1 
    if len(posibles) == 0:
        return -1    
    
    #En casoq ue si hay posibles
    
    for i in posibles:
        aux = px.comparacion(layout.stacks[i])
        if aux > best:
            best = aux
            select = i 
    return select   

In [69]:
def SDpp(layout, s_o, rank, n_elements=100):
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    print(ss_o)
    
    fixed_elements = np.maximum(len(ss_o)-n_elements,0)
    
    while len(layout.stacks[s_o]) > fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            Greedy.fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

In [102]:
import sys
def SFpp(layout, s_d, rank, n):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = Greedy.search_highest(layout, r, ub, s_d)  
            #print(r,ub)
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            
            
            c=Greedy.force_move(layout, s, pos, s_d)
            if c == None: 
                ub=layout.stacks[s][pos]-1
                if ub<=r: 
                    print(layout.stacks, s, pos, s_d)
                    print(str(ub)+ " del stack " +str(s)+ " no se puede volver a colocar en "+str(s_d)+"!!");
                    sys.exit()
            else: 
                ub=r
                cont+=1
                if cont == n: return
                if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

In [103]:
import Greedy as gr
import importlib
importlib.reload(Greedy)

path="Instancias\\10-6\\data10-6-"

nmoves=0
for i in range(1,30):
    nmoves += showtime2(path+ str(i)+".dat")
    
print (nmoves/ 30.0);

[20, 27, 23, 54, 30, 28, 7, 48, 10, 36]
[45, 16, 1, 38, 35, 39, 53, 5, 49, 56, 10, 7]
[42, 43, 21, 3, 55, 2, 50, 29, 57, 1]
[31, 58, 24, 41, 17, 8, 51, 33, 46, 59, 1]
[22, 13, 26, 44, 6, 15, 14, 12]
[34, 37, 18, 19, 4, 60, 5, 7, 15]
136 	 0
[32, 2, 12, 1, 43, 35, 30, 57, 10, 6]
[47, 42, 31, 13, 16, 58, 4, 44, 7, 19, 1]
[60, 14, 48, 33, 34, 38, 11, 46, 17, 49, 59]
[54, 26, 21, 45, 55, 29, 8, 3, 52, 2, 14]
[22, 50, 25, 24, 40, 28, 41, 36, 56]
[5, 9, 51, 39, 20, 53]
126 	 0
[39, 27, 56, 25, 9, 50, 52, 4, 16, 13]
[7, 24, 6, 5, 12, 2, 31, 54, 29, 49, 4, 9]
[17, 26, 1, 59, 20, 36, 45, 21, 55, 53, 7]
[15, 30, 10, 3, 43, 32, 33, 58, 34, 1]
[57, 14, 48, 46, 37, 28, 11, 51]
[60, 42, 23, 41, 44, 8]
128 	 0
[19, 48, 47, 45, 57, 59, 38, 6, 37, 35]
[[59], [34, 33, 29, 41, 51, 10, 36, 52, 55, 56, 6, 38], [1, 24, 12, 18, 23, 13, 11, 53, 32, 49, 57, 19], [3, 22, 28, 25, 7, 5, 8, 54, 50, 58, 37], [27, 4, 40, 17, 39, 9, 46, 30, 42, 2, 35, 48], [21, 43, 14, 31, 60, 16, 20, 15, 26, 44, 45, 47]] 3 -2 0
57 d

SystemExit: 

C:\Users\Dell\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
